In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.utils.data as Data
import torch.nn.init as init
import torch.nn.functional as F

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [3]:
# 加载数据
Data_ = pd.read_csv('../Untitled Folder/3Phrase_FlightData.csv')
Data_.head(5)

,Unnamed: 0,FLIGHT_NO2,FLIGHT_PHASE,ALT_STDC,TAS,GWC,N11,N21,CK_EGT1,N12,...,CK_EGT2,WIN_SPD,WIN_DIR,SAT,PITCH,VRTG,LONG,FF1C,FF2C,FF
0,0,921.0,CLIMB,3796,219.8,64.43,85.4,95.8,718,85.4,...,714,19,253.8,6.5,6.0,0.910,0.2070,3271,3206,6477
1,1,921.0,CLIMB,3800,221.8,64.43,85.5,95.9,719,85.5,...,714,19,253.8,6.5,6.0,0.949,0.2109,3266,3206,6472
2,2,921.0,CLIMB,3808,223.8,64.43,85.8,95.9,719,85.8,...,716,19,253.8,6.3,5.6,0.949,0.2109,3315,3233,6548
3,3,921.0,CLIMB,3816,226.5,64.43,86.0,96.0,721,86.0,...,717,19,253.8,6.3,6.0,0.996,0.2109,3331,3259,6590
4,4,921.0,CLIMB,3828,227.5,64.43,86.1,96.0,721,86.1,...,717,18,251.7,6.3,6.0,1.035,0.2148,3348,3284,6632


In [4]:
# 归一化处理
sc_data = MinMaxScaler(feature_range=(0, 1))
sc_y = MinMaxScaler(feature_range=(0, 1))

data_ = Data_.drop(columns=['Unnamed: 0','FLIGHT_PHASE','FF1C','FF2C'],axis=1).copy()
data_y = Data_[['FF']].copy()
data_sc = sc_data.fit_transform(data_)
sc_y.fit(data_y)
# Train_x = Data.drop(labels=['FF','FF1C','FF2C','Unnamed: 0','FLIGHT_PHASE'],axis=1).copy() 
# train_x = sc_x.transform(Train_x)

# Train_y= Train[['FF']].copy()
# train_y = sc_y.transform(Train_y)

MinMaxScaler()

In [5]:
data_label = data_.columns.values
data_dic = dict(zip(data_label,data_sc.T))
data_df = pd.DataFrame(data_dic)
data_df.head(5)

,FLIGHT_NO2,ALT_STDC,TAS,GWC,N11,N21,CK_EGT1,N12,N22,CK_EGT2,WIN_SPD,WIN_DIR,SAT,PITCH,VRTG,LONG,FF
0,0.0,0.033689,0.038358,0.953144,0.928467,0.887255,0.924242,0.928467,0.889976,0.908861,0.132812,0.707752,0.860979,0.594595,0.205508,0.908910,0.959025
1,0.0,0.033841,0.046030,0.953144,0.929927,0.889706,0.926768,0.929927,0.889976,0.908861,0.132812,0.707752,0.860979,0.594595,0.288136,0.921875,0.958249
2,0.0,0.034146,0.053702,0.953144,0.934307,0.889706,0.926768,0.934307,0.892421,0.913924,0.132812,0.707752,0.857820,0.567568,0.288136,0.921875,0.970045
3,0.0,0.034451,0.064058,0.953144,0.937226,0.892157,0.931818,0.937226,0.894866,0.916456,0.132812,0.707752,0.857820,0.594595,0.387712,0.921875,0.976564
4,0.0,0.034909,0.067894,0.953144,0.938686,0.892157,0.931818,0.938686,0.894866,0.916456,0.125000,0.701896,0.857820,0.594595,0.470339,0.934840,0.983082


In [8]:
# 拆分训练集和测试集，这里以3架航班的数据作为训练集，一架航班的数据作为测试集。
Flight_Number = data_df['FLIGHT_NO2'].unique()
Train = data_df[data_df['FLIGHT_NO2']!=Flight_Number[-1]]
Test = data_df[data_df['FLIGHT_NO2']==Flight_Number[-1]]

In [6]:
def create_seq(input_data, tw):
    inout_seq = []
    F_NO = input_data['FLIGHT_NO2'].unique()
    for f in F_NO:
        in_data = input_data[input_data['FLIGHT_NO2']==f].reset_index(drop=True)
        in_data_x = in_data.drop(columns=['FLIGHT_NO2','FF'],axis=1).copy()
        in_data_y = in_data['FF'].copy()
        L = len(in_data)
        for i in range(L-tw+1):
            train_seq = in_data_x[i:i+tw]
            train_seq = torch.tensor(data=train_seq.values).type(torch.float32).to(device)
            train_label = in_data_y[i+tw-1]
            train_label = torch.tensor(data=train_label).type(torch.float32).to(device)
            inout_seq.append((train_seq ,train_label))
    
    return inout_seq

In [9]:
tw = 15
train_inout_seq = create_seq(Train, tw)
print('The total number of train windows is {}'.format(len(train_inout_seq)))
print('The shape of train_seq is {}'.format(train_inout_seq[0][0].size()))

test_inout_seq = create_seq(Test, tw)
print('The total number of test windows is {}'.format(len(test_inout_seq)))
print('The shape of test_seq is',format(test_inout_seq[0][0].size()))

The total number of train windows is 17897
The shape of train_seq is torch.Size([15, 15])
The total number of test windows is 5311
The shape of test_seq is torch.Size([15, 15])


## CNN-LSTM

In [10]:
class CNN1D(nn.Module):
    def __init__(self,input_dim,output_dim,filter_num,ksize):
        super(CNN1D, self).__init__()
        
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.filter_num = filter_num
        self.ksize = ksize
        self.conv = nn.Conv1d(input_dim,filter_num,ksize)
        self.maxpool = nn.MaxPool1d(2)
        self.linear = nn.Linear(192, self.output_dim)

    def forward(self, x):
        x = x.reshape(-1, self.input_dim, self.input_dim)
        x = self.conv(x)
        x = self.maxpool(x)
        x = x.view(-1,192)
        out = self.linear(x)
        
        return out

In [11]:
# 模型构建
class LSTM(nn.Module):
    def __init__(self, hidden_layer_size = 50, layer_num = 1):
        super(LSTM, self).__init__()
        #填写程序
        self.layer_num = layer_num
        self.input_size = 15
        self.output_size = 15
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(self.input_size,
                           hidden_layer_size,
                           layer_num,
                           batch_first = True,
                           bidirectional = False)
#         self.output = nn.Linear(hidden_layer_size, self.output_size)
#         init_lstm(self.lstm)
        
    def forward(self, input_seq):
        input_seq = input_seq.reshape(-1, self.input_size, self.input_size)

        h0 = torch.zeros(self.layer_num, input_seq.size(0), self.hidden_layer_size).to(device)
        c0 = torch.zeros(self.layer_num, input_seq.size(0), self.hidden_layer_size).to(device)

        out, (hn, cn) = self.lstm(input_seq, (h0, c0))
#         out = self.output(out)
        
        return out       

def init_lstm(x):
    """
    初始化网络权重
    LSTM 网络采tanh激活函数，使用Xavier初始化
    """
    for layer in x._all_weights:
        for w in layer:
            if 'weight' in w:
                init.xavier_normal_(getattr(x, w))

In [12]:
input_dim = len(train_inout_seq[0][0])
output_dim = len(train_inout_seq[0][0])
filter_num = 32
ksize = 4
epo = 2000
batch_size=256

In [58]:
cnn1d = CNN1D(input_dim,output_dim,filter_num,ksize)
lstm = LSTM(hidden_layer_size = 50, layer_num = 1)

In [59]:
def cnn_lstm():
    data_loader = Data.DataLoader(train_inout_seq,batch_size,shuffle=False)
    loss_function = nn.MSELoss(reduction='mean').requires_grad_(True)
    cnn_optimizer = torch.optim.Adam(cnn1d.parameters(), lr=1e-3)
    lstm_optimizer = torch.optim.Adam(lstm.parameters(), lr=1e-3)
    linear_1 = nn.Linear(50,32)
    linear_2 = nn.Linear(32,1)
    
    Losslist = []
    y_pred = []
    for i in range(2000):
        Loss = []
        for batch_id, (seq, label) in enumerate(data_loader):
            cnn_optimizer.zero_grad()
            cnn_out = cnn1d(seq).reshape(len(seq),-1,1)
            lstm_optimizer.zero_grad()
            lstm_out = lstm(seq).permute(0,2,1)
            
            cnnlstm = lstm_out.matmul(cnn_out).permute(0,2,1)
            out = linear_1(cnnlstm)
            out = linear_2(out).squeeze(-1).squeeze(-1)
            loss = loss_function(out, label)
            loss.backward()
            cnn_optimizer.step()
            lstm_optimizer.step()
            
            Loss.append(loss.item())
        L = np.mean(Loss)
        Losslist.append(L)
    
    return Losslist

In [60]:
Losslist = cnn_lstm()

In [49]:
def test():
    loss_function = nn.MSELoss(reduction='mean').requires_grad_(True)
    y_pred_test = []
    y_true_test = []
    pred_loss = []
    linear_1 = nn.Linear(50,32)
    linear_2 = nn.Linear(32,1)
    for i in range(len(test_inout_seq)):
#         label = torch.tensor(test_inout_seq[i][1])
        label = test_inout_seq[i][1]
        
        cnn_out_test = cnn1d(test_inout_seq[i][0]).reshape(1,-1,1)
        lstm_out_test = lstm(test_inout_seq[i][0]).permute(0,2,1)
        cnnlstm = lstm_out_test.matmul(cnn_out_test).permute(0,2,1)
        
        out = linear_1(cnnlstm)
        out = linear_2(out).squeeze(-1).squeeze(-1)
        loss = loss_function(out, label)
        pred_loss.append(loss.item())
        y_pred_test.append(out.item())
        y_true_test.append(label.item())
        
    return y_pred_test,pred_loss
        

In [52]:
y_pred_test,pred_loss = test()